# Implementing protein components
this jnb gives you quick access to the implementation of protein components.


In [ ]:
import gufe

## Dev function

In [ ]:
#input:
pdb_path= "./thrombin_protein.pdb"


In [ ]:
#Imports
import json, ast
from collections import defaultdict

from rdkit import Chem
from rdkit.Chem.rdchem import Mol, Atom, Conformer, EditableMol, BondType

from gufe.components.sub_files.pdbfile import PDBFile #Vendored code - import

from rdkit import Chem
from rdkit.Chem.rdchem import Mol, Atom, Conformer, EditableMol, BondType


In [ ]:
openmm_PDBFile = PDBFile(pdb_path)
name ="trhomb"

In [ ]:
bond_types = {  1 : BondType.SINGLE,
                2 : BondType.DOUBLE,
                3 : BondType.TRIPLE ,
               None :  BondType.SINGLE,
               }

negative_ions = ["CL"]
positive_ions = ["NA", "MG"]

## OpenMM to rdkit

In [ ]:
periodicTable = Chem.GetPeriodicTable()
mol_topology = openmm_PDBFile.topology

rd_mol = Mol()
editable_rdmol = EditableMol(rd_mol)

# Build Topology
_residue_atom_map = defaultdict(list)
histidine_resi_atoms = defaultdict(list)

# Add Atoms
for atom in mol_topology.atoms():
    atomID = int(atom.index)
    resn = atom.residue.name
    resi = int(atom.residue.index)
    chaini = int(atom.residue.chain.index)
    
    
    a = Atom(atom.element.atomic_number)
    a.SetAtomMapNum(atomID)

    a.SetProp("name", atom.name)
    a.SetIntProp("id", atomID)

    a.SetProp("resName", resn)
    a.SetIntProp("resId", resi)
    a.SetIntProp("chainId", chaini)
    
    #For histidine fixes
    if("HIS" ==  atom.residue.name):
        histidine_resi_atoms[str(resi)+"_"+resn].append(atom.name)
    _residue_atom_map[str(resi)+"_"+resn].append(atomID)
    
    editable_rdmol.AddAtom(a)

# Add Bonds
for bond in mol_topology.bonds():
    bond_order = bond_types[bond.order]  
    editable_rdmol.AddBond(beginAtomIdx=bond.atom1.index, endAtomIdx=bond.atom2.index, order=bond_order)    

# Set Positions
# WIP: Make multi frame safe
rd_mol = editable_rdmol.GetMol()
positions = list(map(list, openmm_PDBFile.positions._value))
conf = Conformer(0)
for atom_id, atom_pos in enumerate(positions):
    conf.SetAtomPosition(atom_id, atom_pos) #unit: nm
rd_mol.AddConformer(conf)


# Add Additionals
# Formal Charge
atoms = rd_mol.GetAtoms()
netcharge = 0
for a in atoms:
    atomic_num = a.GetAtomicNum()
    atom_name = a.GetProp("name")
    resn = a.GetProp("resName") 

    connectivity = sum([int(bond.GetBondType()) for bond in a.GetBonds()]) #
    
    default_valence = periodicTable.GetDefaultValence(atomic_num)
    
    # HISTIDINE FIX  resonance
    # Due to the resonance of the Ns in His (which are frequently de/protonating in proteins), there can be bond type changes between ND1-CE1-NE2. 
    if("HIS" == resn and "N" in atom_name and len(atom_name)>1):
        resi = int(a.GetProp("resId"))
        dict_key = str(resi)+"_"+resn

        histidine_atoms = histidine_resi_atoms[dict_key]
        own_prot = atom_name.replace("N", "H") in histidine_atoms
        other_N = list(filter(lambda x: x.startswith("N") and len(x) > 1 and not atom_name== x, histidine_atoms))[0]
        other_prot = other_N.replace("N", "H") in histidine_atoms

        if(own_prot and not other_prot and connectivity != default_valence):
            #change bond-order
            bond_change = [bond for bond in a.GetBonds() if("CE1" in (bond.GetBeginAtom().GetProp("name"),
                                                                    bond.GetEndAtom().GetProp("name")))][0]
            bond_change.SetBondType(bond_types[1])
            
            alternate_atom = [atomB for atomB in rd_mol.GetAtoms() if(atomB.GetProp("resId") == str(resi) and atomB.GetProp("name") == str(other_N))][0]
            bond_change = [bond for bond in alternate_atom.GetBonds() if("CE1" in (bond.GetBeginAtom().GetProp("name"),
                                                                                bond.GetEndAtom().GetProp("name")))][0]
            bond_change.SetBondType(bond_types[2])  
        connectivity = sum([int(bond.GetBondType()) for bond in a.GetBonds()])

    ### HISTIDINE FIX DONE
    
    if(connectivity == 0): #ions:
        if(atom_name in positive_ions):
            fc = default_valence  #e.g. Sodium ions
        elif(atom_name in negative_ions):
            fc = -default_valence  #e.g. Chlorine ions
        else:
            raise ValueError("I don't know this Ion! \t"+atom_name)  
    elif(default_valence > connectivity):
        fc = -(default_valence-connectivity) # negative charge
    elif(default_valence < connectivity):
        fc = +(connectivity-default_valence) # positive charge
    else:
        fc = 0 # neutral

    a.SetFormalCharge(fc)
    a.UpdatePropertyCache(strict=True)
    
    netcharge+=fc

# Molecule props
# Adding nums:
rd_mol.SetProp("ofe-name", name)
rd_mol.SetIntProp("NumAtoms", mol_topology.getNumAtoms())
rd_mol.SetIntProp("NumBonds", mol_topology.getNumBonds())
rd_mol.SetIntProp("NumChains", mol_topology.getNumChains())
rd_mol.SetDoubleProp("NetCharge", netcharge)

# Chains
rd_mol.SetProp("chain_names", str([c.index for c in mol_topology.chains()]))
rd_mol.SetProp("_chain_resi", str([[r.index for r in c.residues()] for c in mol_topology.chains()]))

# Residues
res_seq = " ".join([r.name for r in mol_topology.residues()])
rd_mol.SetProp("sequence", res_seq)
rd_mol.SetProp("_residue_atom_map", str(dict(_residue_atom_map)))

# Box dimensions
pbcVs = list(map(list, mol_topology.getPeriodicBoxVectors()._value)) #unit: nm
unitCellDim = list(map(float, mol_topology.getUnitCellDimensions()._value)) #unit: nm
rd_mol.SetProp("PeriodicBoxVectors", str(pbcVs))
rd_mol.SetProp("UnitCellDimensions", str(unitCellDim))


rd_mol.UpdatePropertyCache(strict=True)

In [ ]:
a.SetB

In [ ]:
openmm_PDBFile.getTopology()

In [ ]:
rd_mol

## Protein Component - Class

### Class Implementation - from_pdb

In [ ]:
import ast, json

from rdkit import Chem
from gufe.components.sub_files.pdbfile import PDBFile
from gufe.components.sub_files.pdbstructure import PdbStructure

from openmm.unit import nanometers, angstroms, is_quantity, norm, Quantity


from gufe import ProteinComponent
pdb_path= "./thrombin_protein.pdb"

prot = ProteinComponent.from_pdbfile(pdb_path, name="thrombin")
prot

In [ ]:
prot._rdkit

In [ ]:
### WIP: to OpenMM/PDB

In [ ]:
pdb_structure = PdbStructure()
openmm_PDBFile = PDBFile(pdb_structure)

periodic_box_vectors = ast.literal_eval(prot._rdkit.GetProp("PeriodicBoxVectors"))
openmm_PDBFile.topology.setPeriodicBoxVectors(None)



## Test

In [ ]:
#WIP

## Class Implementation - dict from and to

In [1]:
from gufe import ProteinComponent
pdb_path= "./thrombin_protein.pdb"

prot = ProteinComponent.from_pdbfile(pdb_path, name="thrombin")
prot

/home/riesbenj/Code/OpenFE/gufe/gufe/components/sub_files/pdbstructure.py:488: UserWarning: WARNING: two consecutive residues with same number (ATOM   4449  N   CYS L   4      10.968  17.388  19.471  1.00 16.59      L    N  , ATOM   4448  HB2 ASP L   4      10.396  20.895  17.001  1.00 39.37      L    H  )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))


ProteinComponent(name=thrombin)

In [5]:
dict_prot = prot.to_dict()
sec_prot = prot.from_dict(dict_prot)
dict_sec_prot = sec_prot.to_dict()

In [3]:
prot.to_rdkit() == sec_prot.to_rdkit()

False

In [4]:
for key, value in dict_prot.items():
    value2 = dict_sec_prot[key]
    print(key, value== value2)
    

atoms True
bonds True
name True
conformers True
molecules True


## Build a bond ordered xml

In [ ]:
import xml.etree.ElementTree as etree

out_path = "../gufe/components/sub_files/data/residues.xml"
in_path = "../gufe/components/sub_files/data/residues_orig.xml"


exception_bond_keys = {
            # AminoAcids
            ##Backbone
            ('C', 'O'): { "order": 2, "resns": "all"},
            
            ## Carbonyls in R
            ("CZ", "NH2"): { "order": 2, "resns": ("ARG")},
            ("CG", "OD1"):{ "order": 2, "resns":  ("ASP", "ASN")},
            ("CD", "OE1"):{ "order": 2, "resns": ("GLN", "GLU")},
            ("CD", "OE"):{ "order": 2, "resns": ("PCA")},
            
            ## Aromatics:
            ("CD2", "CG"):{ "order": 2, "resns": ("HIS")},
            ("CE1", "ND1"):{ "order": 2, "resns": ("HIS")},
            
            ("CG", "CD1"):{ "order": 2, "resns": ("PHE", "TYR", "TRP")},
            ("CE1", "CZ"):{ "order": 2, "resns": ("PHE", "TYR")},
            ("CE2", "CD2"):{ "order": 2, "resns": ("PHE", "TYR")},
            
            ("CD2", "CE3"):{ "order": 2, "resns": ("TRP")},
            ("CE2", "CZ2"):{ "order": 2, "resns": ("TRP")},
            ("CZ3", "CH2"):{ "order": 2, "resns": ("TRP")},

            # NucleicAcids
            ## Phosphates
            ("OP1", "P"):{ "order": 2, "resns": ("U", "G", "A", "C", "DT", "DG", "DC", "DA")},
            
            ## Pyrimidines: Uracil, Thymin and Cytosin
            ("C2", "O2"):{ "order": 2, "resns": ("U", "DT", "C", "DC")},
            ("C5", "C6"):{ "order": 2, "resns": ("U", "DT", "C", "DC")},
            ("C4", "O4"):{ "order": 2, "resns": ("U", "DT")},
            ("C4", "N3"):{ "order": 2, "resns": ("C", "DC")},

            ## Purines: Guanine, Adenine
            ("C2", "N3"):{ "order": 2, "resns": ("G", "DG", "A", "DA")},
            ("C4", "C5"):{ "order": 2, "resns": ("G", "DG", "A", "DA")},
            ("N7", "C8"):{ "order": 2, "resns": ("G", "DG", "A", "DA")},
            ("C6", "O6"):{ "order": 2, "resns": ("G", "DG")},
            ("C6", "N1"):{ "order": 2, "resns": ("A", "DA")},            
             }
#sort keys :
exception_bond_keys = {tuple(sorted(list(key))): value for key, value in exception_bond_keys.items()}
#print(exception_bond_keys)


tree = etree.parse(in_path)

for residue in tree.getroot().findall('Residue'):
    resn = residue.get("name")
    for bond in residue.findall("Bond"):
        c1 = bond.get("from")
        c2 = bond.get("to")
        bond_atoms=tuple(sorted([c1, c2]))
        if(bond_atoms in exception_bond_keys and (exception_bond_keys[bond_atoms]["resns"] == "all" or resn in exception_bond_keys[bond_atoms]["resns"])):
            bond.set("order", str(exception_bond_keys[bond_atoms]["order"]))
        else:
            bond.set("order", str(1))
        #if(resn == "PHE"): print(bond_atoms, bond.get("order"))

tree.write(out_path)